## Vehical Detection project code

In [304]:
import sys
import os.path
sys.path.append('/Users/abadarinath/Applications/anaconda/envs/UdacityNanoCar/lib/python3.5/site-packages')
import numpy as np
import cv2
%matplotlib inline
from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join
import pickle
from skimage.feature import hog
import glob
from sklearn.preprocessing import StandardScaler
import warnings
import sklearn
from sklearn.cross_validation import train_test_split
from random import shuffle
import collections
import time
from scipy.ndimage.measurements import label

warnings.filterwarnings("ignore")
CAMERA_CAL_DIRECTORY = './camera_cal/'

### Function to read all images from directory

In [2]:
def readImagesFromDir(dirPath,fileList=None,rgb=False,format='.jpg'):
    if fileList == None:
        allDirfiles = [dirPath+f for f in listdir(dirPath) if isfile(join(dirPath, f)) and f.endswith(format)]
    else:
        allDirfiles = fileList
    result = []
    for i in range(len(allDirfiles)):
        if rgb:
            bgr_img = cv2.imread(os.path.abspath(allDirfiles[i]))
            b,g,r = cv2.split(bgr_img)       # get b,g,r
            result.append(cv2.merge([r,g,b]))
        else:
            result.append(cv2.imread(os.path.abspath(allDirfiles[i])))
    return result

### function to read the training images recursively in directory. uses the above func to read

In [3]:
def readTrainingImages(dontLoadImg=False):
    vehicles = glob.glob('./sample_training_images/vehicles*/**/*.png', recursive=True)
    nonvehicles = glob.glob('./sample_training_images/non-vehicles*/**/*.png', recursive=True)
    if dontLoadImg:
        return (vehicles,nonvehicles)
    vehiclesImgs = readImagesFromDir(dirPath=None, fileList=vehicles)
    nonvehiclesImgs = readImagesFromDir(dirPath=None, fileList=nonvehicles)
    return (vehiclesImgs,nonvehiclesImgs)
    

### Function to display array of images

In [4]:
def displayImages(imgArray,isGray=False):
    plt.figure()
    for i in range(len(imgArray)):
        if isGray:
            plt.imshow(imgArray[i], cmap='gray')
            plt.show()
        else:
            plt.imshow(imgArray[i])
            plt.show()

### Function to draw boxes of given color on an image

In [5]:
# Define a function that takes an image, a list of bounding boxes, 
# and optional color tuple and line thickness as inputs
# then draws boxes in that color on the output

def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # make a copy of the image
    draw_img = np.copy(img)
    # draw each bounding box on your image copy using cv2.rectangle()
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(draw_img, bbox[0], bbox[1], color, thick)
    # return the image copy with boxes drawn
    return draw_img # Change this line to return image copy with boxes

### Function to return a feature of histograms. It also displays when debugging

In [6]:
# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256), debug=False):
    # Compute the histogram of the RGB channels separately
    rhist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    ghist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    bhist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Generating bin centers
    bin_edges = rhist[1]
    bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((rhist[0], ghist[0], bhist[0]))
    
    # Plot a figure with all three bar charts
    if debug:
        fig = plt.figure(figsize=(12,3))
        plt.subplot(131)
        plt.bar(bin_centers, rhist[0])
        plt.xlim(0, 256)
        plt.title('R Histogram')
        plt.subplot(132)
        plt.bar(bin_centers, ghist[0])
        plt.xlim(0, 256)
        plt.title('G Histogram')
        plt.subplot(133)
        plt.bar(bin_centers, bhist[0])
        plt.xlim(0, 256)
        plt.title('B Histogram')
        fig.tight_layout()
        plt.show()
    # Return the individual histograms, bin_centers and feature vector
    return rhist, ghist, bhist, bin_centers, hist_features

### Function to crop the image for unwanted area.

In [8]:
def cropHorizonInImage(img,debug=False):
    img_size = (img.shape[1], img.shape[0])
    newImg = img[img_size[1]/2:img_size[1], 0:img_size[0]] # Crop from x, y, w, h -> 100, 200, 300, 400
    # NOTE: its img[y: y + h, x: x + w] and *not* img[x: x + w, y: y + h]
    if debug:
        displayImages([newImg])
    return newImg

### Function to return feature bin of image for the given color space

In [9]:
# Define a function to compute color histogram features  
# Pass the color_space flag as 3-letter all caps string
# like 'HSV' or 'LUV' etc.
# KEEP IN MIND IF YOU DECIDE TO USE THIS FUNCTION LATER
# IN YOUR PROJECT THAT IF YOU READ THE IMAGE WITH 
# cv2.imread() INSTEAD YOU START WITH BGR COLOR!
# Define a function to compute color histogram features  
# Pass the color_space flag as 3-letter all caps string
# like 'HSV' or 'LUV' etc.
def bin_spatial(img, color_space='RGB', size=(64, 64), debug=False):
    # Convert image to new color space (if specified)
    cmap=None
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: 
        feature_image = np.copy(img)
    
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(feature_image, size).ravel()
    if debug:
        plt.plot(features)
        plt.show()
    # Return the feature vector
    return features

### Function to return HOG feature for image

In [10]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, debug=False, feature_vec=True):
    if debug:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                       visualise=debug, feature_vector=feature_vec)
    else:
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                   cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=False, 
                   visualise=debug, feature_vector=feature_vec)
    if debug:
        plt.imshow(hog_image, cmap='gray')
        plt.show()
    return features

### Function to extract the feature parameters.

Feature extraction is done on the followin:
 - histogram features of color space
 - BIN_Spatial of RGB, HSV, HLS, YCrCb
 - HOG feature of the gray image
 
 We concatinate all these features and then do a StandardScalar transform to normalize it and return

In [11]:
def extractFeatures(img, debug=False):
    orgImg = img
    grayImg = cv2.cvtColor(orgImg, cv2.COLOR_RGB2GRAY)
    featureList = []
    
    rhist, ghist, bhist, bin_centers, hist_features = color_hist(orgImg,debug=False)
    featureList.append(hist_features)
    featureList.append(bin_spatial(img, color_space='RGB', size=(img.shape[0],img.shape[1]), debug=False))
    featureList.append(bin_spatial(img, color_space='HSV', size=(img.shape[0],img.shape[1]), debug=False))
    featureList.append(bin_spatial(img, color_space='HLS', size=(img.shape[0],img.shape[1]), debug=False))
    featureList.append(bin_spatial(img, color_space='YCrCb', size=(img.shape[0],img.shape[1]), debug=False))
    # Define HOG parameters
    orient = 9
    pix_per_cell = 8
    cell_per_block = 2
    featureList.append(get_hog_features(img=grayImg,orient=orient,pix_per_cell=pix_per_cell,cell_per_block=cell_per_block,debug=False))
    
    if debug:
        for f in featureList:
            print(len(f))
    
    # Create an array stack, NOTE: StandardScaler() expects np.float64
    X = np.concatenate((featureList)).astype(np.float64)
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
    
    if debug:
        print(len(featureList))
        print(len(scaled_X))
        print("memory ",scaled_X.nbytes)
        plt.plot(scaled_X)
        plt.show()
    return scaled_X
    

In [12]:
vehicles,nonvehicle = readTrainingImages(dontLoadImg=True)
# Define the labels vector
y = np.concatenate((np.ones(len(vehicles)), np.zeros(len(nonvehicle))))
final_array = np.dstack((np.concatenate((vehicles,nonvehicle)),y))[0]
np.random.shuffle(final_array)
print(final_array)
print("Total samples", len(final_array), len(y),final_array.shape)
train_samples, test_samples = train_test_split(final_array, test_size=0.1)
tmp = train_samples.shape
print("total training samples",tmp,collections.Counter(train_samples[:,1]))
tmp = test_samples.shape
print("total training samples",tmp,collections.Counter(test_samples[:,1]))

[['./sample_training_images/non-vehicles/myImgs/89790988.png' '0.0']
 ['./sample_training_images/vehicles/KITTI_extracted/2502.png' '1.0']
 ['./sample_training_images/non-vehicles/GTI/image3150.png' '0.0']
 ..., 
 ['./sample_training_images/vehicles/GTI_Far/image0749.png' '1.0']
 ['./sample_training_images/non-vehicles/Extras/extra1902.png' '0.0']
 ['./sample_training_images/non-vehicles/Extras/extra5227.png' '0.0']]
Total samples 22393 22393 (22393, 2)
total training samples (20153, 2) Counter({'0.0': 11153, '1.0': 9000})
total training samples (2240, 2) Counter({'0.0': 1252, '1.0': 988})


In [13]:
X_train = []
y_train = []
for train_sample in train_samples:
    #print(batch_sample)
    bgr_img = cv2.imread(os.path.abspath(train_sample[0]))
    b,g,r = cv2.split(bgr_img)       # get b,g,r
    rgb_img = cv2.merge([r,g,b])
    featuresList = extractFeatures(rgb_img,debug=False)
    X_train.append(featuresList)
    y_train.append([train_sample[1]])


In [14]:
from sklearn.svm import LinearSVC
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')

656.55 Seconds to train SVC...


In [15]:
X_test = []
y_test = []
for test_sample in test_samples:
    #print(batch_sample)
    bgr_img = cv2.imread(os.path.abspath(test_sample[0]))
    b,g,r = cv2.split(bgr_img)       # get b,g,r
    rgb_img = cv2.merge([r,g,b])
    featuresList = extractFeatures(rgb_img,debug=False)
    X_test.append(featuresList)
    y_test.append([test_sample[1]])


In [17]:
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
# for tmp in X_test[0:n_predict]:
#     plt.plot(tmp)
#     plt.show()
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

Test Accuracy of SVC =  0.9621
My SVC predicts:  ['1.0' '0.0' '1.0' '1.0' '0.0' '0.0' '1.0' '1.0' '0.0' '1.0']
For these 10 labels:  [['1.0'], ['0.0'], ['1.0'], ['1.0'], ['0.0'], ['0.0'], ['1.0'], ['1.0'], ['0.0'], ['1.0']]
0.00137 Seconds to predict 10 labels with SVC


In [18]:
import pickle
# save the classifier
with open('my_dumped_classifier.pkl', 'wb') as fid:
    pickle.dump(svc, fid)    


In [19]:

# load it again
with open('my_dumped_classifier.pkl', 'rb') as fid:
    svc = pickle.load(fid)

In [20]:

# Here is your draw_boxes function from the previous exercise
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [21]:
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

In [305]:
iter_cntr = 1

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    heatmap[heatmap > 0] -= 2
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 3
    # Return updated heatmap
    return heatmap# Iterate through list of bboxes

def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    #tmp = np.copy(heatmap)
    heatmap[heatmap <= threshold] = 0
    heatmap[heatmap > threshold+6] = threshold+6
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels,justBoxes=False):
    # Iterate through all detected cars
    on_windows = []
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        if justBoxes == False:
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
        else:
            on_windows.append((bbox[0], bbox[1]))
    # Return the image
    if justBoxes == False:
        return img
    else:
        return on_windows

In [275]:
# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))
#         plt.imshow(test_img)
#         plt.axis('off')
#         plt.show()
        #4) Extract features for that window using single_img_features()
        #5) Scale extracted features to be fed to classifier
        features = extractFeatures(test_img)
        #6) Predict using your classifier
        prediction = clf.predict(features)
#         print(prediction)
        #7) If positive (prediction == 1) then save the window
        if prediction == ['1.0']:
            on_windows.append(window)
#             if window[0][0]<720 and random.randint(0,10) > 8:
#                 cv2.imwrite("./sample_training_images/non-vehicles/myImgs/"+str(random.randint(0,100000000))+".png",test_img)
                
    #8) Return windows for positive detections
    return on_windows

In [218]:
def trainingPipeline(img,debug=False):
    if debug:
        plt.imshow(img)
        plt.show()
    

In [280]:
heat = None
heat_final = None
iter_cntr = 1
def pipeline(img,debug=False):
    
    global heat
    global heat_final
    global iter_cntr
    if heat == None:
        heat = np.zeros_like(img[:,:,0]).astype(np.float)
    if heat_final == None:
        heat_final = np.zeros_like(img[:,:,0]).astype(np.float)
    
    img = unDistortImage(img)
    hot_window = []
    windowSizes = [(64,64),(96,96),(128,128),(160,160),(192,192),(224,224),(256,256)]
    for xyWindow in windowSizes:
        windows = slide_window(img, x_start_stop=[720, None], y_start_stop=[360, 600],xy_window=xyWindow, xy_overlap=(0.5, 0.5))
        hot_window += search_windows(img,windows,svc)
    
#     windows = slide_window(img, x_start_stop=[None, None], y_start_stop=[360, 700],xy_window=(128, 128), xy_overlap=(0.5, 0.5))
#     hot_window += search_windows(img,windows,svc)
    
#     windows = slide_window(img, x_start_stop=[None, None], y_start_stop=[360, 700],xy_window=(192, 192), xy_overlap=(0.5, 0.5))
#     hot_window += search_windows(img,windows,svc)
    
    
#     windows = slide_window(img, x_start_stop=[None, None], y_start_stop=[360, 700],xy_window=(256, 256), xy_overlap=(0.5, 0.5))
#     hot_window += search_windows(img,windows,svc)
    
    # Add heat to each box in box list
    heat = add_heat(heat,hot_window)
    # Apply threshold to help remove false positives
    heat_thresh = apply_threshold(heat,2)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat_thresh, 0, 255)
#     plt.imshow(heat,cmap='hot')
#     plt.show()
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(img), labels,justBoxes=False)
#     heatMapBoxes = draw_labeled_bboxes(np.copy(img), labels,justBoxes=True)
    
    result = draw_img#draw_boxes(img, hot_window, color=(0, 0, 255), thick=6) 
    
#     plt.imshow(heatmap_final,cmap='hot')
#     plt.show()
    
#     if debug:
#         print("Orignial image")
#         displayImages([img])
#         print("final result")
#         displayImages([result])
#         print("********************************************************************************************************")
        
    return result

In [289]:
heat = None
heat_final = None
testImgs = readImagesFromDir('./test_images/',rgb=True,format='.png')
heat = None
for i in range(1):#len(testImgs)):
    img = testImgs[i]
    result = pipeline(img, True)

In [306]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import random
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    result = None
    result = pipeline(image)
    return result

In [ ]:
heat = None #reset
white_output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4





  0%|          | 0/1261 [00:00<?, ?it/s]


  0%|          | 1/1261 [00:00<08:04,  2.60it/s]


  0%|          | 2/1261 [00:00<08:03,  2.60it/s]


  0%|          | 3/1261 [00:01<07:59,  2.63it/s]


  0%|          | 4/1261 [00:01<07:55,  2.64it/s]


  0%|          | 5/1261 [00:01<07:54,  2.64it/s]


  0%|          | 6/1261 [00:02<07:51,  2.66it/s]


  1%|          | 7/1261 [00:02<07:51,  2.66it/s]


  1%|          | 8/1261 [00:03<07:53,  2.65it/s]


  1%|          | 9/1261 [00:03<08:19,  2.51it/s]


  1%|          | 10/1261 [00:03<08:19,  2.50it/s]


  1%|          | 11/1261 [00:04<08:15,  2.52it/s]


  1%|          | 12/1261 [00:04<08:21,  2.49it/s]


  1%|          | 13/1261 [00:05<08:10,  2.54it/s]


  1%|          | 14/1261 [00:05<08:15,  2.51it/s]


  1%|          | 15/1261 [00:05<08:10,  2.54it/s]


  1%|▏         | 16/1261 [00:06<08:29,  2.45it/s]


  1%|▏         | 17/1261 [00:06<08:18,  2.50it/s]


  1%|▏         | 18/1261 [00:07<08:07,  2.55it/s]


  2%|▏         | 19/1261 [0

In [ ]:
import pygame
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))